# Lab 2  - Image Processing

### Names:
#### Spencer Bernardo-Cheng, Rob Keehan, Max Goldstein

In [19]:
import pandas as pd
import numpy as np
import glob
import cv2
# stack overflow example taken from: https://stackoverflow.com/questions/37747021/create-numpy-array-of-images
X_data = []
Y_data = []
files = glob.glob ("./cell_images/Parasitized/*.png")
for myFile in files:
    image = cv2.imread(myFile,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(135,135))
    X_data.append(image)
    Y_data.append(0)
files = glob.glob ("./cell_images/Uninfected/*.png")
for myFile in files:
    image = cv2.imread(myFile,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(135,135))
    X_data.append (image)
    Y_data.append(1)
npArr =np.array(X_data)
print('X_data shape:', np.array(X_data).shape)

X_data shape: (27558, 135, 135, 3)


In [20]:
print(X_data[0].shape())
print(X_data[1].shape())
pixels = npArr.flatten()
pixels.shape()

TypeError: 'tuple' object is not callable